#### Simple function to transform 2D lat/lon arrays into 1D CSV files

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
import os

from scipy.spatial import distance

In [19]:

def csv_lat_lon(src_fl):
    ''' Transform lat/lon arrays into 1D CSV files '''
    src_dir = '/Users/nberg/projects/doe/data/processed/'
    dest_dir = '/Users/nberg/projects/doe/data/invariant/'
    
    df_out = pd.DataFrame()
    
    src_nc = netCDF4.Dataset(src_dir + src_fl)
 
    if len(src_nc['lat'].shape) == 2:
        lat2d = src_nc['lat'][:,:]
        lon2d = src_nc['lon'][:,:]       
        
    elif len(src_nc['lat'].shape) == 1:
        # Create a 2D meshgrid from the 1D lat/lons, then flatten for 1D pts
        lon1d = src_nc['lon'][:] - 180.
        lat1d = src_nc['lat'][:]
        lon2d, lat2d = np.meshgrid(lon1d, lat1d)
        
    df_out['lat'] = lat2d.flatten()
    df_out['lon'] = lon2d.flatten()
    src_nc.close()

    # Obtain the prefix for creating the destination file
    str_parts = src_fl.split('_')
    if str_parts[0] in ['wrf', 'livneh', 'narr']:
            prefix = str_parts[0]
    else:
        prefix = '-'.join([str_parts[0], str_parts[1]])
    
    dest_fl = dest_dir+prefix+'_lat_lon.csv'
    if os.path.exists(dest_fl): os.remove(dest_fl)
    
    df_out.to_csv(dest_fl, float_format = '%.4f')
    print(dest_fl)


if __name__=='__main__':
#     files = ['wrf_hist_1991-2000_tmax_april.nc', 
#             'loca_wrf_hist_1991-2000_tmax_april.nc',
#             'livneh_hist_1991-2000_tmax_april.nc',
#             'loca_livneh_hist_1991-2000_tmax_april.nc']

    #files = ['narr_hist_1991-2000_tmax_april.nc']
    files = ['cnrm_cm5_hist_1981-2000_tmax_april.nc']
    
    for src_fl in files:
        csv_lat_lon(src_fl)



/Users/nberg/projects/doe/data/invariant/cnrm-cm5_lat_lon.csv


#### Generic nearest neighbor function to locate a dataset;s grid cell from the lat/lon data contained in the joined CSV files

In [20]:
def nearest_neighbor(inv_fl, pt_lat, pt_lon):
    """ Locate nearest grid cell given a point lat/lon """

    ncfile = netCDF4.Dataset(inv_fl, 'r')
    lat = ncfile.variables['lat'][:,:]
    lon = ncfile.variables['lon'][:,:]
    nlat, nlon = np.shape(lat)
    ncfile.close()

    # set up 2D lat/lon coordinate array 
    npts = nlat*nlon
    ds_coords = np.zeros([2,npts])
    ds_indices = np.zeros([2,npts])
    idx = 0 
    for i in range(nlat):
        for j in range(nlon):
            ds_coords[0,idx] = lat[i,j]
            ds_coords[1,idx] = lon[i,j]
            ds_indices[0,idx] = i 
            ds_indices[1,idx] = j 
            idx += 1

    # Compute distances from point coordinate to all dataset grid cells
    obs_pt = np.array([pt_lat,pt_lon]).reshape((2,1))
    dist = distance.cdist(ds_coords.T,obs_pt.T)

    near_idx = dist.argmin()
    near_lat, near_lon = ds_coords[0,near_idx], ds_coords[1,near_idx]
    near_lat_idx, near_lon_idx = int(ds_indices[0,near_idx]), int(ds_indices[1,near_idx])

    return near_lat_idx, near_lon_idx

In [23]:
def nearest_neighbor_1d(inv_fl, pt_lat, pt_lon):
    """ Locate nearest grid cell given a point lat/lon """

    ncfile = netCDF4.Dataset(inv_fl, 'r')
    lat = ncfile.variables['lat'][:]
    lon = ncfile.variables['lon'][:]
    nlat = len(lat)
    nlon = len(lon)
    ncfile.close()

    # set up 2D lat/lon coordinate array 
    npts = nlat*nlon
    ds_coords = np.zeros([2,npts])
    ds_indices = np.zeros([2,npts])
    idx = 0 
    for i in range(nlat):
        for j in range(nlon):
            ds_coords[0,idx] = lat[i]
            ds_coords[1,idx] = lon[j]
            ds_indices[0,idx] = i 
            ds_indices[1,idx] = j 
            idx += 1

    # Compute distances from point coordinate to all dataset grid cells
    obs_pt = np.array([pt_lat,pt_lon]).reshape((2,1))
    dist = distance.cdist(ds_coords.T,obs_pt.T)

    near_idx = dist.argmin()
    near_lat, near_lon = ds_coords[0,near_idx], ds_coords[1,near_idx]
    near_lat_idx, near_lon_idx = int(ds_indices[0,near_idx]), int(ds_indices[1,near_idx])

    return near_lat_idx, near_lon_idx

#### Create new netcdf files storing a CA mask array

In [22]:
def create_mask(dataset):
    """ Create watershed masks and output as a netCDF file """

    # Read in the invariant file to get dimension sizes
    src_path = '/Users/nberg/projects/doe/data/invariant/'
    
    ds_inv_fl = src_path+'{0}_lat_lon.nc'.format(dataset)
    ds_join_fl = src_path+'{0}_CA_mask_join.csv'.format(dataset)
    
    ds_inv_nc = netCDF4.Dataset(ds_inv_fl, 'r')
    lat2d = ds_inv_nc.variables['lat'][:,:]
    lon2d = ds_inv_nc.variables['lon'][:,:]
    ds_inv_nc.close()

    nlat, nlon = np.shape(lat2d)

    # Read in the joined CSV file from QGIS
    df = pd.read_csv(ds_join_fl).drop(['field_1', 'REGION', 'DIVISION', 'STATEFP', 'STATENS',
                                   'GEOID', 'STUSPS', 'NAME', 'LSAD', 'MTFCC', 'FUNCSTAT',
                                   'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'], axis=1)
    
    # Generate a 2D array of the watershed codes 
    mask_arr = np.zeros([nlat,nlon], dtype='int')
    for idx, row in df.iterrows():
        if idx % 1000 == 0:
            print(idx)
        near_lat_idx, near_lon_idx = nearest_neighbor(ds_inv_fl, df['lat'][idx], df['lon'][idx])
        mask_arr[near_lat_idx,near_lon_idx] = 1

    # Output masks in a new netCDF file 
    fl_out = src_path+'{0}_CA_mask.nc'.format(dataset)
    if os.path.exists(fl_out):
        os.remove(fl_out)

    ncfile_out = netCDF4.Dataset(fl_out, 'w')
    ncfile_out.createDimension('lat', nlat)
    ncfile_out.createDimension('lon', nlon)
    lat_out = ncfile_out.createVariable('latitude', 'f4', ('lat', 'lon'),)
    lon_out = ncfile_out.createVariable('longitude', 'f4', ('lat', 'lon'),)
    mask_out = ncfile_out.createVariable('landmask', 'int', ('lat', 'lon',))
    setattr(lat_out, 'units', 'degrees_north')
    setattr(lon_out, 'units', 'degrees_east')
    setattr(mask_out, 'description', 'California_landmask=1')
    lat_out[:,:] = lat2d[:,:]
    lon_out[:,:] = lon2d[:,:]
    mask_out[:,:] = mask_arr[:,:]
    ncfile_out.close()
    print(fl_out)
    
if __name__ == '__main__':
    #datasets = ['wrf', 'livneh', 'loca-wrf', 'loca-livneh']
    #datasets = ['wrf', 'loca-wrf', 'loca-livneh']
    datasets = ['narr']
    for dataset in datasets:
        ret = create_mask(dataset)

0
/Users/nberg/projects/doe/data/invariant/narr_CA_mask.nc


In [28]:
def create_mask_1d(dataset):
    """ Create watershed masks and output as a netCDF file """

    # Read in the invariant file to get dimension sizes
    src_path = '/Users/nberg/projects/doe/data/invariant/'
    
    ds_inv_fl = src_path+'{0}_lat_lon.nc'.format(dataset)
    ds_join_fl = src_path+'{0}_CA_mask_join.csv'.format(dataset)
    
    ds_inv_nc = netCDF4.Dataset(ds_inv_fl, 'r')
    lat = ds_inv_nc.variables['lat'][:]
    lon = ds_inv_nc.variables['lon'][:]
    ds_inv_nc.close()

    nlat = len(lat)
    nlon = len(lon)

    # Read in the joined CSV file from QGIS
    df = pd.read_csv(ds_join_fl).drop(['field_1', 'REGION', 'DIVISION', 'STATEFP', 'STATENS',
                                   'GEOID', 'STUSPS', 'NAME', 'LSAD', 'MTFCC', 'FUNCSTAT',
                                   'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'], axis=1)
    df['lon'] = df['lon'] + 360.
    
    # Generate a 2D array of the watershed codes 
    mask_arr = np.zeros([nlat,nlon], dtype='int')
    for idx, row in df.iterrows():
        if idx % 1000 == 0:
            print(idx)
        near_lat_idx, near_lon_idx = nearest_neighbor_1d(ds_inv_fl, df['lat'][idx], df['lon'][idx])
        mask_arr[near_lat_idx,near_lon_idx] = 1

    # Output masks in a new netCDF file 
    fl_out = src_path+'{0}_CA_mask.nc'.format(dataset)
    if os.path.exists(fl_out):
        os.remove(fl_out)

    ncfile_out = netCDF4.Dataset(fl_out, 'w')
    ncfile_out.createDimension('lat', nlat)
    ncfile_out.createDimension('lon', nlon)
    lat_out = ncfile_out.createVariable('latitude', 'f4', ('lat'),)
    lon_out = ncfile_out.createVariable('longitude', 'f4', ('lon'),)
    mask_out = ncfile_out.createVariable('landmask', 'int', ('lat', 'lon',))
    setattr(lat_out, 'units', 'degrees_north')
    setattr(lon_out, 'units', 'degrees_east')
    setattr(mask_out, 'description', 'California_landmask=1')
    lat_out[:] = lat[:]
    lon_out[:] = lon[:]
    mask_out[:,:] = mask_arr[:,:]
    ncfile_out.close()
    print(fl_out)
    
if __name__ == '__main__':
    datasets = ['cnrm-cm5']
    for dataset in datasets:
        ret = create_mask_1d(dataset)

0
/Users/nberg/projects/doe/data/invariant/cnrm-cm5_CA_mask.nc
